Before we get started, a couple of reminders to keep in mind when using iPython notebooks:

- Remember that you can see from the left side of a code cell when it was last run if there is a number within the brackets.
- When you start a new notebook session, make sure you run all of the cells up to the point where you last left off. Even if the output is still visible from when you ran the cells in your previous session, the kernel starts in a fresh state so you'll need to reload the data, etc. on a new session.
- The previous point is useful to keep in mind if your answers do not match what is expected in the lesson's quizzes. Try reloading the data and run all of the processing steps one by one in order to make sure that you are working with the same variables and data that are at each quiz stage.


## Load Data from CSVs

In [1]:
import unicodecsv

## Longer version of code (replaced with shorter, equivalent version below)

# enrollments = []
# f = open('enrollments.csv', 'rb')
# reader = unicodecsv.DictReader(f)
# for row in reader:
#     enrollments.append(row)
# f.close()
def readcsv(filename):
    with open(filename, 'rb') as f:
        reader = unicodecsv.DictReader(f)
        return list(reader)

enrollments = readcsv('enrollments.csv')
print enrollments[0]
# for i in range(50):
#     if(enrollments[i]['status'] == 'current'):
#         print enrollments[i]

{u'status': u'canceled', u'is_udacity': u'True', u'is_canceled': u'True', u'join_date': u'2014-11-10', u'account_key': u'448', u'cancel_date': u'2015-01-14', u'days_to_cancel': u'65'}


In [2]:
#####################################
#                 1                 #
#####################################

daily_engagement = readcsv('daily_engagement.csv')
project_submissions = readcsv('project_submissions.csv')
print daily_engagement[0]
print project_submissions[0]

{u'lessons_completed': u'0.0', u'num_courses_visited': u'1.0', u'total_minutes_visited': u'11.6793745', u'projects_completed': u'0.0', u'acct': u'0', u'utc_date': u'2015-01-09'}
{u'lesson_key': u'3176718735', u'processing_state': u'EVALUATED', u'account_key': u'256', u'assigned_rating': u'UNGRADED', u'completion_date': u'2015-01-16', u'creation_date': u'2015-01-14'}


## Fixing Data Types

In [3]:
from datetime import datetime as dt

# Takes a date as a string, and returns a Python datetime object. 
# If there is no date given, returns None
def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, '%Y-%m-%d')
#         return date
    
# Takes a string which is either an empty string or represents an integer,
# and returns an int or None.
def parse_maybe_int(i):
    if i == '':
        return None
    else:
        return int(i)

# Clean up the data types in the enrollments table
for enrollment in enrollments:
    enrollment['cancel_date'] = parse_date(enrollment['cancel_date'])
    enrollment['days_to_cancel'] = parse_maybe_int(enrollment['days_to_cancel'])
    enrollment['is_canceled'] = enrollment['is_canceled'] == 'True'
    enrollment['is_udacity'] = enrollment['is_udacity'] == 'True'
    enrollment['join_date'] = parse_date(enrollment['join_date'])
    
enrollments[0]

{u'account_key': u'448',
 u'cancel_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'days_to_cancel': 65,
 u'is_canceled': True,
 u'is_udacity': True,
 u'join_date': datetime.datetime(2014, 11, 10, 0, 0),
 u'status': u'canceled'}

In [4]:
# Clean up the data types in the engagement table
for engagement_record in daily_engagement:
    engagement_record['lessons_completed'] = int(float(engagement_record['lessons_completed']))
    engagement_record['num_courses_visited'] = int(float(engagement_record['num_courses_visited']))
    engagement_record['projects_completed'] = int(float(engagement_record['projects_completed']))
    engagement_record['total_minutes_visited'] = float(engagement_record['total_minutes_visited'])
    engagement_record['utc_date'] = parse_date(engagement_record['utc_date'])
    
daily_engagement[0]

{u'acct': u'0',
 u'lessons_completed': 0,
 u'num_courses_visited': 1,
 u'projects_completed': 0,
 u'total_minutes_visited': 11.6793745,
 u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}

In [5]:
# Clean up the data types in the submissions table
for submission in project_submissions:
    submission['completion_date'] = parse_date(submission['completion_date'])
    submission['creation_date'] = parse_date(submission['creation_date'])

project_submissions[0]

{u'account_key': u'256',
 u'assigned_rating': u'UNGRADED',
 u'completion_date': datetime.datetime(2015, 1, 16, 0, 0),
 u'creation_date': datetime.datetime(2015, 1, 14, 0, 0),
 u'lesson_key': u'3176718735',
 u'processing_state': u'EVALUATED'}

Note when running the above cells that we are actively changing the contents of our data variables. If you try to run these cells multiple times in the same session, an error will occur.

## Investigating the Data

In [6]:
#####################################
#                 3                 #
#####################################

## Rename the "acct" column in the daily_engagement table to "account_key".


for d in daily_engagement:
    d['account_key'] = d['acct']
    del d['acct']

In [7]:
#####################################
#                 2                 #
#####################################
def unique_students(record):
    unique = set()
    for n in record:
        unique.add(n['account_key'])
    return unique

print len(enrollments)
print len(unique_students(enrollments))

print len(daily_engagement)
print len(unique_students(daily_engagement))

print len(project_submissions)
print len(unique_students(project_submissions))

# enrollment_unique = {};
# for e in enrollments:
#     enrollment_unique[e['account_key']] = True
# print len(enrollment_unique)

## Find the total number of rows and the number of unique students (account keys)
## in each table.

1640
1302
136240
1237
3642
743


## Problems in the Data

In [8]:
#####################################
#                 3                 #
#####################################
print daily_engagement[0]['account_key']

## Rename the "acct" column in the daily_engagement table to "account_key".

0


## Missing Engagement Records

In [9]:
#####################################
#                 4                 #
#####################################
unique_engagement_students = unique_students(daily_engagement)
unique_enrollment_students = unique_students(enrollments)

for e in enrollments:
    student = e['account_key']
    if(not(student in unique_engagement_students)):
        print e
        break
## Find any one student enrollments where the student is missing from the daily engagement table.
## Output that enrollment.

{u'status': u'canceled', u'is_udacity': False, u'is_canceled': True, u'join_date': datetime.datetime(2014, 11, 12, 0, 0), u'account_key': u'1219', u'cancel_date': datetime.datetime(2014, 11, 12, 0, 0), u'days_to_cancel': 0}


## Checking for More Problem Records

In [10]:
#####################################
#                 5                 #
#####################################
missing = set()
for e in enrollments:
    student = e['account_key']
    if(not(student in unique_engagement_students)):
        if(e['days_to_cancel'] != 0):
            missing.add(student)
            print e
print len(missing)
## Find the number of surprising data points (enrollments missing from
## the engagement table) that remain, if any.

{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 1, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 3, 10, 0, 0), u'days_to_cancel': 59}
{u'status': u'canceled', u'is_udacity': True, u'is_canceled': True, u'join_date': datetime.datetime(2015, 3, 10, 0, 0), u'account_key': u'1304', u'cancel_date': datetime.datetime(2015, 6, 17, 0, 0), u'days_to_cancel': 99}
{u'status': u'current', u'is_udacity': True, u'is_canceled': False, u'join_date': datetime.datetime(2015, 2, 25, 0, 0), u'account_key': u'1101', u'cancel_date': None, u'days_to_cancel': None}
2


## Tracking Down the Remaining Problems

In [11]:
# Create a set of the account keys for all Udacity test accounts
udacity_test_accounts = set()
for enrollment in enrollments:
    if enrollment['is_udacity']:
        udacity_test_accounts.add(enrollment['account_key'])
len(udacity_test_accounts)

6

In [12]:
# Given some data with an account_key field, removes any records corresponding to Udacity test accounts
def remove_udacity_accounts(data):
    non_udacity_data = []
    for data_point in data:
        if data_point['account_key'] not in udacity_test_accounts:
            non_udacity_data.append(data_point)
    return non_udacity_data

In [13]:
# Remove Udacity test accounts from all three tables
non_udacity_enrollments = remove_udacity_accounts(enrollments)
non_udacity_engagement = remove_udacity_accounts(daily_engagement)
non_udacity_submissions = remove_udacity_accounts(project_submissions)

print len(non_udacity_enrollments)
print len(non_udacity_engagement)
print len(non_udacity_submissions)

1622
135656
3634


## Refining the Question

In [14]:
#####################################
#                 6                 #
#####################################

## Create a dictionary named paid_students containing all students who either
## haven't canceled yet or who remained enrolled for more than 7 days. The keys
## should be account keys, and the values should be the date the student enrolled.

paid_students = {}
for e in non_udacity_enrollments:
    if(e['days_to_cancel'] > 7 or e['days_to_cancel'] == None):
        acct_key = e['account_key']
        enroll_date = e['join_date']
        if(acct_key not in paid_students or enroll_date > paid_students[acct_key]):
            paid_students[acct_key] = enroll_date
#         if(e['enrollment_date'] - d)

len(paid_students)
        

995

## Getting Data from First Week

In [35]:
# Takes a student's join date and the date of a specific engagement record,
# and returns True if that engagement record happened within one week
# of the student joining.
def within_one_week(join_date, engagement_date):
    time_delta = engagement_date - join_date
    return time_delta.days < 7 and time_delta.days >= 0

def remove_free_trial_cancels(data):
    new_data = []
    for data_point in data:
        if data_point['account_key'] in paid_students:
            new_data.append(data_point)
    return new_data

In [36]:
#####################################
#                 7                 #
#####################################

paid_enrollments = remove_free_trial_cancels(non_udacity_enrollments)
paid_engagement = remove_free_trial_cancels(non_udacity_engagement)
paid_submissions = remove_free_trial_cancels(non_udacity_submissions)

print len(paid_enrollments)
print len(paid_engagement)
print len(paid_submissions)

## Create a list of rows from the engagement table including only rows where
## the student is one of the paid students you just found, and the date is within
## one week of the student's join date.

paid_engagement_in_first_week = []
non_udacity_engagement[0]
for e in non_udacity_engagement:
    if(e['account_key'] in paid_students):
        j_date = paid_students[e['account_key']]
        e_date = e['utc_date']
        if(within_one_week(j_date, e_date)):
           paid_engagement_in_first_week.append(e)
print len(paid_engagement_in_first_week)
# for i in range(10):
#     print paid_engagement_in_first_week[i]

1293
134549
3618
6919


## Exploring Student Engagement

In [37]:
from collections import defaultdict

# Create a dictionary of engagement grouped by student.
# The keys are account keys, and the values are lists of engagement records.
def group_data(data, key_name):
    grouped_data = defaultdict(list)
    for data_point in data:
        key = data_point[key_name]
        grouped_data[key].append(data_point)
    return grouped_data

In [38]:
# Create a dictionary with the total minutes each student spent in the classroom during the first week.
# The keys are account keys, and the values are numbers (total minutes)

def sum_grouped_data(grouped_data, field_name):
    summed_data = {}
    for key, data_points in grouped_data.items():
        total = 0
        for data_point in data_points:
            total += data_point[field_name]
        summed_data[key] = total
    return summed_data

In [39]:
import numpy as np

# Summarize the data about minutes spent in the classroom
def print_summary(values):
    print 'Mean:', np.mean(values)
    print 'Standard deviation:', np.std(values)
    print 'Minimum:', np.min(values)
    print 'Maximum:', np.max(values)

## Debugging Data Analysis Code

## Lessons Completed in First Week

In [40]:
#####################################
#                 9                 #
#####################################

## Adapt the code above to find the mean, standard deviation, minimum, and maximum for
## the number of lessons completed by each student during the first week. Try creating
## one or more functions to re-use the code above.

engagement_by_account = group_data(paid_engagement_in_first_week, 'account_key')
total_minutes_by_account = sum_grouped_data(engagement_by_account, 'total_minutes_visited')
total_lessons_by_account = sum_grouped_data(engagement_by_account, 'lessons_completed')
    
#Print these results
total_minutes = total_minutes_by_account.values()
print_summary(total_minutes)
total_lessons = total_lessons_by_account.values()
print_summary(total_lessons)

Mean: 306.708326753
Standard deviation: 412.996933409
Minimum: 0.0
Maximum: 3564.7332645
Mean: 1.63618090452
Standard deviation: 3.00256129983
Minimum: 0
Maximum: 36


In [41]:
#####################################
#                 8                 #
#####################################
for acct in total_minutes_by_account:
    if(total_minutes_by_account[acct] > 100):
        print "OMG HERES A NEW ONE!:"
        print acct
        print paid_students[acct]
        for e in engagement_by_account[acct]:
            print e
            print '\n'
            break
    break
#         print engagement_by_account[acct]
## Go through a similar process as before to see if there is a problem.
## Locate at least one surprising piece of data, output it, and take a look at it.

OMG HERES A NEW ONE!:
619
2015-06-06 00:00:00
{u'lessons_completed': 2, u'num_courses_visited': 3, 'has_visited': 1, u'total_minutes_visited': 156.649419833, u'projects_completed': 0, 'account_key': u'619', u'utc_date': datetime.datetime(2015, 6, 6, 0, 0)}




## Number of Visits in First Week

In [42]:
######################################
#                 10                 #
######################################

## Find the mean, standard deviation, minimum, and maximum for the number of
## days each student visits the classroom during the first week.
for acct, engagements in engagement_by_account.items():
    for e in engagements:
        if e['num_courses_visited'] > 0:
            e['has_visited'] = 1
        else:
            e['has_visited'] = 0

total_visits_by_account = sum_grouped_data(engagement_by_account, 'has_visited')
total_visits = total_visits_by_account.values()
print_summary(total_visits)





Mean: 2.86733668342
Standard deviation: 2.25519800292
Minimum: 0
Maximum: 7


## Splitting out Passing Students

In [48]:
######################################
#                 11                 #
######################################

## Create two lists of engagement data for paid students in the first week.
## The first list should contain data for students who eventually pass the
## subway project, and the second list should contain data for students
## who do not.
subway_project_lesson_keys = ['746169184', '3176718735']

# paid_engagements_by_account = group_data(paid_submissions, 'account_key')

pass_subway_project = set()
for s in paid_submissions:
    project = s['lesson_key']
    rating = s['assigned_rating']
    if(project in subway_project_lesson_keys and (rating == 'PASSED' or rating == 'DISTINCTION')):
        pass_subway_project.add(s['account_key'])

print len(pass_subway_project)
submissions_by_account = group_data(non_udacity_submissions, 'account_key')


passing_engagement = []
non_passing_engagement = []

print paid_engagement_in_first_week[0]

for engagement in paid_engagement_in_first_week:
    acct = engagement['account_key']
    if(acct in pass_subway_project):
        passing_engagement.append(engagement)
    else:
        non_passing_engagement.append(engagement)

print len(passing_engagement)
print len(non_passing_engagement)

647
{u'lessons_completed': 0, u'num_courses_visited': 1, 'has_visited': 1, u'total_minutes_visited': 11.6793745, u'projects_completed': 0, 'account_key': u'0', u'utc_date': datetime.datetime(2015, 1, 9, 0, 0)}
4527
2392


## Comparing the Two Student Groups

In [44]:
######################################
#                 12                 #
######################################

## Compute some metrics you're interested in and see how they differ for
## students who pass the subway project vs. students who don't. A good
## starting point would be the metrics we looked at earlier (minutes spent
## in the classroom, lessons completed, and days visited).

## Making Histograms

In [25]:
######################################
#                 13                 #
######################################

## Make histograms of the three metrics we looked at earlier for both
## students who passed the subway project and students who didn't. You
## might also want to make histograms of any other metrics you examined.

## Improving Plots and Sharing Findings

In [26]:
######################################
#                 14                 #
######################################

## Make a more polished version of at least one of your visualizations
## from earlier. Try importing the seaborn library to make the visualization
## look better, adding axis labels and a title, and changing one or more
## arguments to the hist() function.